# Job Listing Portal — Fix & Test Notebook

This notebook documents reproducible steps to lint, test and patch the Job Listing Portal. It contains: environment setup, static checks, jsdom/Jest tests, and programmatic patches for common JavaScript issues in `script/`.

Usage notes:
- Open this notebook in VS Code/Jupyter or read it as a plan of actions.
- Commands target a Node/npm environment (Windows PowerShell).

## 1) Setup dev environment

Commands to run in a PowerShell terminal (run from project root `c:\Users\bhaskar1\Desktop\JoB2.0`):

```powershell
# initialize package.json
npm init -y
# install dev tooling
npm install --save-dev eslint jest jsdom @babel/preset-env @babel/core @babel/register

# optional: add scripts to package.json via npm set-script or manually edit package.json
# example package.json scripts:
# "scripts": {
#   "lint": "eslint script --ext .js",
#   "test": "jest"
# }
```

Create minimal config files below (the notebook will show suggested contents to write into `.eslintrc.json` and `jest.config.js`).

### Write config files example

Create `.eslintrc.json` (recommended minimal config):

```json
{
  "env": { "browser": true, "es2021": true, "node": true },
  "extends": "eslint:recommended",
  "parserOptions": { "ecmaVersion": 12, "sourceType": "module" },
  "rules": {
    "no-unused-vars": ["warn"],
    "no-undef": "error",
    "semi": ["error", "always"]
  }
}
```

Create `jest.config.js` for using jsdom and babel (so tests can import the code):

```javascript
module.exports = {
  testEnvironment: 'jsdom',
  transform: {
    '^.+\\.js$': ['babel-jest', { presets: ['@babel/preset-env'] }]
  }
};
```

You can write these files using PowerShell `Set-Content` or manually create them in the project root.

## 2) Run linters & auto-fix (ESLint)

Commands to run (PowerShell):

```powershell
# Run ESLint and show problems
npx eslint script --ext .js

# Attempt automatic fixes
npx eslint script --ext .js --fix
```

Notes:
- ESLint will surface style issues and some potential errors (unused vars, duplicate declarations). Keep fixes focused — don't automatically fix logic bugs.
- Use the output to direct targeted code patches in subsequent cells.

## 3) Detect duplicate function definitions & quick static checks

Use this Node script (save as `tools/find_duplicates.js` or run via `node -e "..."`) to list top-level `function` declarations and duplicates across `script/`:

```javascript
const fs = require('fs');
const path = require('path');
const dir = path.join(__dirname, '..', 'script');
const files = fs.readdirSync(dir).filter(f => f.endsWith('.js'));
const funcMap = {};
for (const f of files) {
  const src = fs.readFileSync(path.join(dir,f),'utf8');
  const re = /function\s+([A-Za-z0-9_]+)\s*\(/g;
  let m;
  while ((m = re.exec(src))) {
    const name = m[1];
    funcMap[name] = funcMap[name] || [];
    funcMap[name].push(f);
  }
}
for (const [name, arr] of Object.entries(funcMap)) {
  if (arr.length > 1) console.log(`${name}: ${arr.join(', ')}`);
}
```

Run it to find duplicates and then consolidate (e.g., `updateNavUI` often appears in both `auth.js` and `main.js`).

## 4) Unit tests with jsdom (Jest) — fixtures & examples

Create a test file `__tests__/main.spec.js` with tests for `displayJobListings`, `searchJobs`, `applyForJob`, and `simulateLogin`.

Example test skeleton (use `jest` + `jsdom`):

```javascript
/** @jest-environment jsdom */

// load scripts in correct order
require('../script/dashboard.js');
require('../script/auth.js');
require('../script/main.js');

describe('Job Portal basic behaviors', () => {
  beforeEach(() => {
    document.body.innerHTML = `
      <div id="job-listings"></div>
      <div id="toasts"></div>
      <button id="nav-login"></button>
      <button id="nav-logout"></button>
      <button id="nav-dashboard"></button>
      <section id="job-search-section"></section>
      <section id="auth-form-section"></section>
      <section id="dashboard-section"></section>
    `;
    // reset auth
    localStorage.clear();
  });

  test('displayJobListings inserts job cards', () => {
    const sample = [{id:999, title:'Test Role', company:'X', location:'Remote', salary:'10k-20k', description:'x'}];
    displayJobListings(sample);
    expect(document.getElementById('job-listings').querySelectorAll('.job-card').length).toBe(1);
  });

  test('searchJobs filters results', () => {
    window.jobs = [ {id:1,title:'Python Dev',description:'work',location:'Remote'},{id:2,title:'Designer',description:'design',location:'NY'} ];
    document.getElementById('keyword-search')?.remove();
    const input = document.createElement('input'); input.id='keyword-search'; input.value='python'; document.body.appendChild(input);
    const loc = document.createElement('select'); loc.id='location-filter'; document.body.appendChild(loc);
    searchJobs();
    expect(document.getElementById('job-listings').textContent.toLowerCase()).toContain('python');
  });

  test('simulateLogin sets localStorage and updates nav', () => {
    simulateLogin('alice','pwd');
    expect(localStorage.getItem('isAuthenticated')).toBe('true');
  });
});
```

Run with `npx jest --runInBand`.

Notes:
- Tests load source files via `require`. If source files assume DOM on load, some functions may run immediately; ensure minimal DOM nodes exist in `beforeEach` to prevent errors.

## 5) Fix duplicate `updateNavUI`

If `find_duplicates.js` reports `updateNavUI` exists in multiple files (e.g., `auth.js` and `main.js`), consolidate to a single canonical implementation in `script/main.js` and remove or call it from `auth.js`.

Suggested patch steps (manual or scripted):

- Keep implementation in `script/main.js` and delete the duplicate function in `script/auth.js`.
- If `auth.js` calls `updateNavUI`, ensure `main.js` is loaded before `auth.js` in `index.html` or export a small `utils.js` to host shared helpers.

Example Node snippet to remove a duplicate block in `auth.js` (use cautiously):

```javascript
const fs = require('fs');
let s = fs.readFileSync('./script/auth.js','utf8');
// naive removal by function name (make a backup first)
s = s.replace(/function updateNavUI\([\s\S]*?\}\n\n/, '\n');
fs.writeFileSync('./script/auth.js', s, 'utf8');
```

After consolidation, run ESLint and the tests to confirm nothing else broke.

## 6) Fix saveJobListing ID generation

Problem: creating new jobs using `id: jobs.length + 1` can cause collisions when jobs are deleted or when initial IDs are not sequential. Fix by using max existing id + 1.

Patch example to apply to `script/dashboard.js` (search for where new job objects are created):

```javascript
// old
// id: jobs.length + 1

// new
const maxId = jobs && jobs.length ? Math.max(...jobs.map(j => j.id)) : 0;
const newId = maxId + 1;
// use newId as id for the created job
```

Add a unit test that creates then deletes a job and creates another, asserting unique IDs are assigned.

## 7) Fix employer → company mapping in `viewMyListings`

Problem: employer accounts may store company mapping under keys like `companyProfile:${companyName}` or `employerCompany:${username}`. When displaying an employer's listings, resolve the company name from storage fallback to username.

Suggested patch for `viewMyListings` or `renderEmployerDashboard`:

```javascript
// resolve company for this employer
const username = localStorage.getItem('userName');
const employerCompany = localStorage.getItem('employerCompany:' + username) || localStorage.getItem('companyProfile:' + username) || username;
// filter jobs by employerCompany
const myJobs = jobs.filter(j => j.company === employerCompany);
```

Add tests: simulate employer saving a company mapping, create a job with that company name, then `viewMyListings` should show it.

## 8) Defensive checks & small runtime fixes

Add null/undefined guards in code that assumes DOM nodes or stored values exist. Examples to apply in `dashboard.js`, `main.js`:

- Check `if (!jobs) jobs = [];` before operations.
- In `viewCompanyProfile`, guard before parsing dates:

```javascript
const profileRaw = localStorage.getItem('companyProfile:' + companyName);
const profile = profileRaw ? JSON.parse(profileRaw) : null;
if (profile && profile.updatedAt) {
  const dt = new Date(profile.updatedAt);
  // use dt safely
}
```

- When reading elements: `const el = document.getElementById('foo'); if (el) el.doSomething();`

Make these small, localized edits, then re-run ESLint and tests.

## 9) Integration test: login → apply → verify

Integration test pseudocode (Jest + jsdom):

```javascript
/** @jest-environment jsdom */
require('../script/dashboard.js');
require('../script/auth.js');
require('../script/main.js');

test('jobseeker can login and apply to a job', () => {
  document.body.innerHTML = `<div id="job-listings"></div>`;
  // ensure a known job exists
  window.jobs = [ {id: 42, title:'Test Role', company:'X', location:'Remote', salary:'10k', description:'' , applications: [] } ];
  // register/login
  simulateRegister('bob@example.com','pw');
  simulateLogin('bob@example.com','pw');
  // simulate apply
  applyForJob(42);
  const username = localStorage.getItem('userName');
  expect(username).toBeTruthy();
  // appliedJobs may be stored in global or localStorage depending on implementation
  expect(appliedJobs[username] || []).toContain(42);
  // job object should include application entry
  expect(window.jobs.find(j=>j.id===42).applications.length).toBeGreaterThan(0);
});
```

Run with `npx jest --runInBand` and iterate on failures.

## 10) Apply patches & run test suite

Suggested commands to apply patches (PowerShell examples):

```powershell
# 1) Run ESLint and attempt auto-fix
npx eslint script --ext .js --fix

# 2) Run tests (Jest)
npx jest --runInBand

# 3) If a patch is created programmatically, show diff before commit
git init  # if repo not initialized
git add -A
git commit -m "wip: pre-fix commit"
# apply programmatic changes via node scripts or manual edits
git diff --staged > ../job_portal_fixes.patch
```

When tests fail, inspect stack traces, update source files (the notebook earlier contains example patch snippets). After all tests pass, create a reviewable patch via `git format-patch` or `git diff` and commit.

---

If you want, I can now apply a small set of concrete fixes to `script/main.js` to:
- make the title animator safe (store/clear interval id),
- add `startTitleAnimator`/`stopTitleAnimator` helpers,
- add `disableAnimations` / `enableAnimations` helpers (apply `html.no-animations`),

and then create a small patch for `script/dashboard.js` to use robust ID generation for new jobs. Tell me if you want me to apply these edits automatically now.